In [1]:
import pandas as pd
from datetime import datetime
import re
from sklearn.utils import shuffle

FUNCTIONS USED FOR DATA PREPROCESSING

In [4]:

def extract_numeric_value(value_string):
    # Use regular expression to find digits and commas
    matches = re.findall(r'\d+', value_string.replace(',', ''))

    # Join the matches into a single string and convert to float
    numeric_value = float(''.join(matches))
    return numeric_value

def convert_aed_to_usd(amount_in_aed):
    amount_in_aed = extract_numeric_value(amount_in_aed)
    amount_in_usd = amount_in_aed * 0.27
    return amount_in_usd

def convert_price(df):
    for index, row in df.iterrows():
        # Access individual elements using column names
        row['Price'] = convert_aed_to_usd(row['Price'])




In [18]:
def clean_arrival_time(arrival_time):
    if '+' in arrival_time:
        return arrival_time.split('+')[0]
    else:
        return arrival_time

In [5]:
def convert_to_minutes(duration):
    parts = duration.split(' ')
    
    hours = 0
    minutes = 0
    
    if 'h' in parts[0]:
        hours = int(parts[0].replace('h', ''))
    
    if 'm' in parts[-1]:
        minutes = int(parts[-1].replace('m', ''))
    
    return hours * 60 + minutes

In [6]:
def convert_stops_to_numeric(df, column_name):
    mapping_dict = {value: index for index, value in enumerate(df[column_name].unique())}

    df[column_name] = df[column_name].replace(mapping_dict)

    return df

In [2]:
def categorize_time(hour):
    # Split the string using ":" as the delimiter
    hours, minutes = map(int, hour.split(':'))
    # Extract the hour part as an integer
    hour_as_int = int(hours)
    if 4 <= hour_as_int < 7:
        return "Early Morning"
    elif 7 <= hour_as_int < 12:
        return "Morning"
    elif 12 <= hour_as_int < 17:
        return "Afternoon"
    elif 17 <= hour_as_int < 20:
        return "Evening"
    elif 20 <= hour_as_int < 24:
        return "Night"
    else:
        return "Late Night"

IMPORTING THE DATA SCRAPPED

In [8]:
data= pd.read_csv('../mydata/airlines_dataset.csv')
data


,Airline,Source,Destination,Duration,stops,class,depature time,arrival time,Price,Date
0,TUI Fly Belgium,CMN,PAR,3h 25m,direct,economy,07:20,11:45,AED 563,2024-05-29
1,Royal Air Maroc,CMN,PAR,2h 55m,direct,economy,13:25,17:20,AED 977,2024-05-29
2,Royal Air Maroc,CMN,PAR,2h 50m,direct,economy,18:35,22:25,"AED 1,102",2024-05-29
3,Royal Air Maroc,CMN,PAR,2h 50m,direct,economy,12:20,16:10,"AED 1,085",2024-05-29
4,Royal Air Maroc,CMN,PAR,2h 50m,direct,economy,14:15,18:05,"AED 1,102",2024-05-29
...,...,...,...,...,...,...,...,...,...,...
24960,Korean Air,NRT,IST,32h 20m,1 stop,business,17:20,19:40+1,"AED 7,970",2024-07-06
24961,LOT,NRT,IST,20h 30m,1 stop,business,22:35,13:05+1,"AED 12,710",2024-07-06
24962,China Southern,NRT,IST,23h 05m,2 stops,business,13:25,06:30+1,"AED 12,070",2024-07-06
24963,Etihad Airways,NRT,IST,27h 40m,1 stop,business,17:30,15:10+1,"AED 10,715",2024-07-06


DATA CLEANING, PREPROCESSING AND FEATURE ENGINEERING

In [9]:
#checking null values 
data.isnull().sum()

Airline          0
Source           0
Destination      0
Duration         0
stops            0
class            0
depature time    0
arrival time     0
Price            0
Date             0
dtype: int64

In [10]:
convert_price(data)

In [11]:
data

,Airline,Source,Destination,Duration,stops,class,depature time,arrival time,Price,Date
0,TUI Fly Belgium,CMN,PAR,3h 25m,direct,economy,07:20,11:45,152.01,2024-05-29
1,Royal Air Maroc,CMN,PAR,2h 55m,direct,economy,13:25,17:20,263.79,2024-05-29
2,Royal Air Maroc,CMN,PAR,2h 50m,direct,economy,18:35,22:25,297.54,2024-05-29
3,Royal Air Maroc,CMN,PAR,2h 50m,direct,economy,12:20,16:10,292.95,2024-05-29
4,Royal Air Maroc,CMN,PAR,2h 50m,direct,economy,14:15,18:05,297.54,2024-05-29
...,...,...,...,...,...,...,...,...,...,...
24960,Korean Air,NRT,IST,32h 20m,1 stop,business,17:20,19:40+1,2151.9,2024-07-06
24961,LOT,NRT,IST,20h 30m,1 stop,business,22:35,13:05+1,3431.7,2024-07-06
24962,China Southern,NRT,IST,23h 05m,2 stops,business,13:25,06:30+1,3258.9,2024-07-06
24963,Etihad Airways,NRT,IST,27h 40m,1 stop,business,17:30,15:10+1,2893.05,2024-07-06


In [12]:
data['Duration'] = data['Duration'].apply(convert_to_minutes)

In [13]:
data

,Airline,Source,Destination,Duration,stops,class,depature time,arrival time,Price,Date
0,TUI Fly Belgium,CMN,PAR,205,direct,economy,07:20,11:45,152.01,2024-05-29
1,Royal Air Maroc,CMN,PAR,175,direct,economy,13:25,17:20,263.79,2024-05-29
2,Royal Air Maroc,CMN,PAR,170,direct,economy,18:35,22:25,297.54,2024-05-29
3,Royal Air Maroc,CMN,PAR,170,direct,economy,12:20,16:10,292.95,2024-05-29
4,Royal Air Maroc,CMN,PAR,170,direct,economy,14:15,18:05,297.54,2024-05-29
...,...,...,...,...,...,...,...,...,...,...
24960,Korean Air,NRT,IST,1940,1 stop,business,17:20,19:40+1,2151.9,2024-07-06
24961,LOT,NRT,IST,1230,1 stop,business,22:35,13:05+1,3431.7,2024-07-06
24962,China Southern,NRT,IST,1385,2 stops,business,13:25,06:30+1,3258.9,2024-07-06
24963,Etihad Airways,NRT,IST,1660,1 stop,business,17:30,15:10+1,2893.05,2024-07-06


In [14]:
data = convert_stops_to_numeric(data, 'stops')

C:\Users\Zbook\AppData\Local\Temp\ipykernel_22072\3184267031.py:4: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[column_name] = df[column_name].replace(mapping_dict)


In [15]:
data

,Airline,Source,Destination,Duration,stops,class,depature time,arrival time,Price,Date
0,TUI Fly Belgium,CMN,PAR,205,0,economy,07:20,11:45,152.01,2024-05-29
1,Royal Air Maroc,CMN,PAR,175,0,economy,13:25,17:20,263.79,2024-05-29
2,Royal Air Maroc,CMN,PAR,170,0,economy,18:35,22:25,297.54,2024-05-29
3,Royal Air Maroc,CMN,PAR,170,0,economy,12:20,16:10,292.95,2024-05-29
4,Royal Air Maroc,CMN,PAR,170,0,economy,14:15,18:05,297.54,2024-05-29
...,...,...,...,...,...,...,...,...,...,...
24960,Korean Air,NRT,IST,1940,1,business,17:20,19:40+1,2151.9,2024-07-06
24961,LOT,NRT,IST,1230,1,business,22:35,13:05+1,3431.7,2024-07-06
24962,China Southern,NRT,IST,1385,2,business,13:25,06:30+1,3258.9,2024-07-06
24963,Etihad Airways,NRT,IST,1660,1,business,17:30,15:10+1,2893.05,2024-07-06


In [19]:
data['arrival time'] = data['arrival time'].apply(clean_arrival_time)
data['arrival time']= data['arrival time'].apply(categorize_time)
data['depature time']= data['depature time'].apply(categorize_time)

In [22]:
data['Date'] = data['Date'].str.strip()
data['Date'] = pd.to_datetime(data['Date'], format='%Y-%m-%d')
data['Price'] = data['Price'].astype(float)

In [23]:
data.dtypes


Airline                  object
Source                   object
Destination              object
Duration                  int64
stops                     int64
class                    object
depature time            object
arrival time             object
Price                   float64
Date             datetime64[ns]
dtype: object

ADDING NEW FEATURE OF NUMBER OF DAYS LEFT FOR THE FLIGHT

In [24]:
search_date = pd.to_datetime('2024-05-28')

# Calculate the days left
data['Days Left'] = (data['Date'] - search_date).dt.days

In [25]:
data

,Airline,Source,Destination,Duration,stops,class,depature time,arrival time,Price,Date,Days Left
0,TUI Fly Belgium,CMN,PAR,205,0,economy,Morning,Morning,152.01,2024-05-29,1
1,Royal Air Maroc,CMN,PAR,175,0,economy,Afternoon,Evening,263.79,2024-05-29,1
2,Royal Air Maroc,CMN,PAR,170,0,economy,Evening,Night,297.54,2024-05-29,1
3,Royal Air Maroc,CMN,PAR,170,0,economy,Afternoon,Afternoon,292.95,2024-05-29,1
4,Royal Air Maroc,CMN,PAR,170,0,economy,Afternoon,Evening,297.54,2024-05-29,1
...,...,...,...,...,...,...,...,...,...,...,...
24960,Korean Air,NRT,IST,1940,1,business,Evening,Evening,2151.90,2024-07-06,39
24961,LOT,NRT,IST,1230,1,business,Night,Afternoon,3431.70,2024-07-06,39
24962,China Southern,NRT,IST,1385,2,business,Afternoon,Early Morning,3258.90,2024-07-06,39
24963,Etihad Airways,NRT,IST,1660,1,business,Evening,Afternoon,2893.05,2024-07-06,39


ADDING NEW FEATURE OF DAY OF THE WEEK

In [26]:
data['Day of Week'] = data['Date'].dt.dayofweek


In [27]:
data

,Airline,Source,Destination,Duration,stops,class,depature time,arrival time,Price,Date,Days Left,Day of Week
0,TUI Fly Belgium,CMN,PAR,205,0,economy,Morning,Morning,152.01,2024-05-29,1,2
1,Royal Air Maroc,CMN,PAR,175,0,economy,Afternoon,Evening,263.79,2024-05-29,1,2
2,Royal Air Maroc,CMN,PAR,170,0,economy,Evening,Night,297.54,2024-05-29,1,2
3,Royal Air Maroc,CMN,PAR,170,0,economy,Afternoon,Afternoon,292.95,2024-05-29,1,2
4,Royal Air Maroc,CMN,PAR,170,0,economy,Afternoon,Evening,297.54,2024-05-29,1,2
...,...,...,...,...,...,...,...,...,...,...,...,...
24960,Korean Air,NRT,IST,1940,1,business,Evening,Evening,2151.90,2024-07-06,39,5
24961,LOT,NRT,IST,1230,1,business,Night,Afternoon,3431.70,2024-07-06,39,5
24962,China Southern,NRT,IST,1385,2,business,Afternoon,Early Morning,3258.90,2024-07-06,39,5
24963,Etihad Airways,NRT,IST,1660,1,business,Evening,Afternoon,2893.05,2024-07-06,39,5


In [28]:
data = shuffle(data, random_state=42)
data

,Airline,Source,Destination,Duration,stops,class,depature time,arrival time,Price,Date,Days Left,Day of Week
21361,Air France,CMN,NRT,1945,2,economy,Evening,Morning,1082.70,2024-07-03,36,2
5050,Egypt Air,IST,CMN,1275,1,economy,Early Morning,Night,350.19,2024-06-07,10,4
20280,Royal Air Maroc,IST,CMN,295,0,economy,Morning,Afternoon,603.72,2024-07-01,34,0
6481,SWISS,PAR,NRT,965,1,business,Morning,Morning,4137.21,2024-06-09,12,6
17212,Ajet,LAX,IST,1670,3,economy,Morning,Night,775.17,2024-06-27,30,3
...,...,...,...,...,...,...,...,...,...,...,...,...
21575,Etihad Airways,NRT,PAR,1275,1,economy,Evening,Morning,755.73,2024-07-03,36,2
5390,Qatar Airways,IST,NRT,1010,1,business,Night,Evening,3707.10,2024-06-07,10,4
860,Cathay Pacific,NRT,CMN,1950,2,economy,Afternoon,Afternoon,1030.05,2024-05-30,2,3
15795,Singapore Airlines,NRT,PAR,1645,1,business,Morning,Morning,2691.09,2024-06-24,27,0


SAVING THE PREPROCESSED DATA TO A CSV FILE TO PREFORM EXPLORATORY DATA ANALYSIS

In [29]:
data.to_csv("../mydata/preprocessed_data.csv", index=False)
